In [1]:
import warnings
# warnings.filterwarnings("ignore")

In [2]:
import datetime
import glob
from functools import lru_cache, partial
from pprint import pprint
import os

import gym_trading_env
import gymnasium as gym
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from gym_trading_env.downloader import download, EXCHANGE_LIMIT_RATES

from tqdm.autonotebook import tqdm
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import datetime
from Keys import oanda_paper_api_key

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_5400/9826573.py:16: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:

EXCHANGE_LIMIT_RATES["binanceus"] = {
    "limit" : 200, # One request will query 200 data points (aka candlesticks).
    "pause_every" : 120, # it will pause every 120 requests.
    "pause" : 2, # the pause will last 2 seconds.
}

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


In [4]:

EXCHANGE_LIMIT_RATES["binanceus"] = {
    "limit" : 200, # One request will query 200 data points (aka candlesticks).
    "pause_every" : 120, # it will pause every 120 requests.
    "pause" : 2, # the pause will last 2 seconds.
}

In [5]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH',
            ]
FX_PAIRS=['EUR_USD','AUD_USD','GBP_USD','USD_JPY','NZD_USD','USD_CAD',
            'USD_CHF','USD_MXN',
            ]
target_pair='EUR_USD'
time_frame="1h"

In [6]:

# download(exchange_names = ["binanceus"],
#     symbols= tqdm(COIN_PAIRS),
#     timeframe= time_frame,
#     dir = "data",
#     since= datetime.datetime(year= 2022, month= 1, day=1),
# )

In [7]:
from oandapyV20.contrib.factories import InstrumentsCandlesFactory

In [8]:
def download_forex(symbols,
                   timeframe,
                    dir = "forex_data/",
                    since= datetime.datetime(year= 2022, month= 1, day=1),

                   ):
        api=API(access_token=oanda_paper_api_key,environment='practice')

        if not os.path.exists(dir):

            os.makedirs(dir+'/')
        now=datetime.datetime.now()

        start_date=since.strftime('%Y-%m-%dT%H:%M:%SZ')
        now=now.strftime('%Y-%m-%dT%H:%M:%SZ')

        params={
            "granularity":timeframe,
            "from": start_date,
            "to": now,
        }
        for symbol in symbols:
            klines=InstrumentsCandlesFactory(instrument=symbol, params=params)
            candles=[]
            for batch in klines:
                resp = api.request(batch)
                
                for candle in resp.get('candles'):

                    ctime = candle.get('time')[0:19]
                    
                    try:
                        row = dict(
                            date_close=ctime,
                            open=candle['mid']['o'],
                            high=candle['mid']['h'],
                            low=candle['mid']['l'],
                            close=candle['mid']['c'],
                            volume=candle['volume'],
                        )
                    
                        candles.append(row)
                    except Exception as e:

                        print(e, klines)
                    
            candles=pd.DataFrame(candles)
            print(symbol,len(candles))
            candles.to_pickle(f"{dir}/Oanda-{symbol}-{time_frame}.pkl")

In [9]:

download_forex(
    symbols= tqdm(FX_PAIRS),
    timeframe= 'H1',
    dir = "data_forex",
    since= datetime.datetime(year= 2022, month= 1, day=1),
)

  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:05<00:38,  5.55s/it]

EUR_USD 15533


 25%|██▌       | 2/8 [00:10<00:31,  5.27s/it]

AUD_USD 15528


...

USD_CAD 15534


 88%|████████▊ | 7/8 [00:34<00:04,  4.88s/it]

USD_CHF 15529


100%|██████████| 8/8 [00:39<00:00,  4.95s/it]

USD_MXN 15529
